In [1]:
import cobra
import numpy as np
import warnings 
warnings.filterwarnings("ignore")
#GEM_Recon2_thermocurated_redHUMAN = cobra.io.load_matlab_model('GEM_Recon2_thermocurated_redHUMAN.mat')
#cobra.io.save_json_model(GEM_Recon2_thermocurated_redHUMAN, "GEM_Recon2_thermocurated_redHUMAN.json")
normal = cobra.io.load_json_model("GEM_Recon2_thermocurated_redHUMAN.json")
print(
normal.reactions.get_by_id("PHETHPTOX2").bounds,
normal.reactions.get_by_id("r0399").bounds)

(0.0, 100.0) (-100.0, 100.0)


In [4]:
import pandas as pd

frozen         = normal.metabolites.get_by_id("tyr_L_e").reactions.union(normal.metabolites.get_by_id("tyr_L_c").reactions).union(normal.metabolites.get_by_id("tyr_L_m").reactions)
tyr_L_ids    = [list(frozen)[i].id for i in range(0,len(frozen))]
tyr_L_names  = [list(frozen)[i].name for i in range(0,len(frozen))]
tyr_L_rxns   = [list(frozen)[i].reaction for i in range(0,len(frozen))]

list_of_tuples   = list(zip(tyr_L_ids,tyr_L_names, tyr_L_rxns))  
columns          = ["tyr_L_e_ids","tyr_L_e_names","tyr_L_e_rxns"]
tyr_L          =  pd.DataFrame(list_of_tuples, columns = columns)  


frozen         = normal.metabolites.get_by_id("phe_L_e").reactions.union(normal.metabolites.get_by_id("phe_L_c").reactions).union(normal.metabolites.get_by_id("phe_L_m").reactions)
phe_L_ids    = [list(frozen)[i].id for i in range(0,len(frozen))]
phe_L_names  = [list(frozen)[i].name for i in range(0,len(frozen))]
phe_L_rxns   = [list(frozen)[i].reaction for i in range(0,len(frozen))]

list_of_tuples   = list(zip(phe_L_ids,phe_L_names, phe_L_rxns))  
columns          = ["phe_L_e_ids","phe_L_e_names","phe_L_e_rxns"]
phe_L          =  pd.DataFrame(list_of_tuples, columns = columns)

## FBA

In [5]:
sol_normal = normal.optimize()

normal.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
3mob_e,EX_3mob_e,0.1277,5,0.28%
5mthf_e,EX_5mthf_e,0.008531,20,0.07%
6dhf_e,EX_6dhf_e,0.4125,44,7.95%
HC00250_e,EX_HC00250_e,0.1489,0,0.00%
HC01610_e,EX_HC01610_e,0.02919,36,0.46%
Rtotal_e,EX_Rtotal_e,0.05592,1,0.02%
adn_e,EX_adn_e,2.812,10,12.32%
alaala_e,EX_alaala_e,0.8085,6,2.13%
arg_L_e,EX_arg_L_e,1,6,2.63%
asn_L_e,EX_asn_L_e,1,4,1.75%


In [6]:
normal.metabolites.phe_L_c.summary()

Percent,Flux,Reaction,Definition
100.00%,0.8298,PHEtec,phe_L_e <=> phe_L_c
Percent,Flux,Reaction,Definition
100.00%,-0.8298,biomass,0.505626 ala_L_c + 0.35926 arg_L_c + 0.279425 asn_L_c + 0.352607 asp_L_c + 20.704451 atp_c + 0.020401 chsterol_c + 0.011658 clpn_hs_c + 0.039036 ctp_c + 0.046571 cys_L_c + 0.013183 datp_n + 0.009442 dctp_n + 0.009898 dgtp_n + 0.013091 dttp_n + 0.275194 g6p_c + 0.325996 gln_L_c + 0.385872 glu_L_c + 0.538891 gly_c + 0.036117 gtp_c + 20.650823 h2o_c + 0.126406 his_L_c + 0.286078 ile_L_c + 0.545544 leu_L_c + 0.592114 lys_L_c + 0.153018 met_L_c + 0.023315 pail_hs_c + 0.154463 pchol_hs_c + 0.055374 pe_hs_c + 0.002914 pglyc_hs_c + 0.259466 phe_L_c + 0.412484 pro_L_c + 0.005829 ps_hs_c + 0.392525 ser_L_c + 0.017486 sphmyln_hs_c + 0.31269 thr_L_c + 0.013306 trp_L_c + 0.159671 tyr_L_c + 0.053446 utp_c + 0.352607 val_L_c --> 20.650823 adp_c + 20.650823 h_c + 20.650823 pi_c


In [7]:
normal.metabolites.tyr_L_c.summary()

Percent,Flux,Reaction,Definition
100.00%,0.5106,TYRt,tyr_L_e <=> tyr_L_c
Percent,Flux,Reaction,Definition
100.00%,-0.5106,biomass,0.505626 ala_L_c + 0.35926 arg_L_c + 0.279425 asn_L_c + 0.352607 asp_L_c + 20.704451 atp_c + 0.020401 chsterol_c + 0.011658 clpn_hs_c + 0.039036 ctp_c + 0.046571 cys_L_c + 0.013183 datp_n + 0.009442 dctp_n + 0.009898 dgtp_n + 0.013091 dttp_n + 0.275194 g6p_c + 0.325996 gln_L_c + 0.385872 glu_L_c + 0.538891 gly_c + 0.036117 gtp_c + 20.650823 h2o_c + 0.126406 his_L_c + 0.286078 ile_L_c + 0.545544 leu_L_c + 0.592114 lys_L_c + 0.153018 met_L_c + 0.023315 pail_hs_c + 0.154463 pchol_hs_c + 0.055374 pe_hs_c + 0.002914 pglyc_hs_c + 0.259466 phe_L_c + 0.412484 pro_L_c + 0.005829 ps_hs_c + 0.392525 ser_L_c + 0.017486 sphmyln_hs_c + 0.31269 thr_L_c + 0.013306 trp_L_c + 0.159671 tyr_L_c + 0.053446 utp_c + 0.352607 val_L_c --> 20.650823 adp_c + 20.650823 h_c + 20.650823 pi_c


In [9]:
phe_L_fluxes   = sol_normal.fluxes[phe_L_ids].to_frame() #.query('abs(fluxes) > 1e-7', inplace=True)
phe_L_fluxes.query('abs(fluxes) > 1e-7', inplace=True)
phe_L_fluxes = phe_L_fluxes.rename_axis('phe_L_e_ids').reset_index()
pd.merge(phe_L_fluxes, phe_L)

,phe_L_e_ids,fluxes,phe_L_e_names,phe_L_e_rxns
0,RE0938E,0.829787,RE0938,CE2917_e + h2o_e <=> CE4724_e + phe_L_e
1,biomass,3.198056,Generic human biomass reaction,0.505626 ala_L_c + 0.35926 arg_L_c + 0.279425 ...
2,r1626,0.829787,Amino Acid-Polyamine-Organocation (APC) TCDB:2...,ile_L_c + phe_L_e <=> ile_L_e + phe_L_c


In [10]:
tyr_L_fluxes   = sol_normal.fluxes[tyr_L_ids].to_frame()  #.query('abs(fluxes) > 1e-7', inplace=True)
tyr_L_fluxes.query('abs(fluxes) > 1e-7', inplace=True)
tyr_L_fluxes = tyr_L_fluxes.rename_axis('tyr_L_e_ids').reset_index()
pd.merge(tyr_L_fluxes, tyr_L)

,tyr_L_e_ids,fluxes,tyr_L_e_names,tyr_L_e_rxns
0,EX_tyr_L_e,-0.510637,L-Tyrosine exchange,tyr_L_e <=>
1,TYRt,2.255317,L-tyrosine transport,tyr_L_e <=> tyr_L_c
2,r1634,-1.744680,Amino Acid-Polyamine-Organocation (APC) TCDB:2...,ile_L_c + tyr_L_e <=> ile_L_e + tyr_L_c
3,biomass,3.198056,Generic human biomass reaction,0.505626 ala_L_c + 0.35926 arg_L_c + 0.279425 ...


## Parsimonious FBA

In [11]:
pfba_solution = cobra.flux_analysis.pfba(normal)
sol_normal = pfba_solution

In [13]:
phe_L_fluxes   = sol_normal.fluxes[phe_L_ids].to_frame() #.query('abs(fluxes) > 1e-7', inplace=True)
phe_L_fluxes.query('abs(fluxes) > 1e-7', inplace=True)
phe_L_fluxes = phe_L_fluxes.rename_axis('phe_L_e_ids').reset_index()
pd.merge(phe_L_fluxes, phe_L)

,phe_L_e_ids,fluxes,phe_L_e_names,phe_L_e_rxns
0,EX_phe_L_e,-0.829787,exchange reaction for L-phenylalanine,phe_L_e <--
1,PHEtec,0.829787,L-phenylalanine transport via diffusion (extra...,phe_L_e <=> phe_L_c
2,biomass,3.198056,Generic human biomass reaction,0.505626 ala_L_c + 0.35926 arg_L_c + 0.279425 ...


In [14]:
tyr_L_fluxes   = sol_normal.fluxes[tyr_L_ids].to_frame()  #.query('abs(fluxes) > 1e-7', inplace=True)
tyr_L_fluxes.query('abs(fluxes) > 1e-7', inplace=True)
tyr_L_fluxes = tyr_L_fluxes.rename_axis('tyr_L_e_ids').reset_index()
pd.merge(tyr_L_fluxes, tyr_L)

,tyr_L_e_ids,fluxes,tyr_L_e_names,tyr_L_e_rxns
0,EX_tyr_L_e,-0.510637,L-Tyrosine exchange,tyr_L_e <=>
1,TYRt,0.510637,L-tyrosine transport,tyr_L_e <=> tyr_L_c
2,biomass,3.198056,Generic human biomass reaction,0.505626 ala_L_c + 0.35926 arg_L_c + 0.279425 ...


## geometric FBA

In [15]:
geometric_fba_sol = cobra.flux_analysis.geometric_fba(normal, epsilon=1e-01, max_tries=50)
sol_normal = geometric_fba_sol
phe_L_fluxes   = sol_normal.fluxes[phe_L_e_ids].to_frame() #.query('abs(fluxes) > 1e-7', inplace=True)
phe_L_fluxes.query('abs(fluxes) > 1e-7', inplace=True)
phe_L_fluxes = phe_L_fluxes.rename_axis('phe_L_e_ids').reset_index()
pd.merge(phe_L_fluxes, phe_L)

Process ForkPoolWorker-68:
Process ForkPoolWorker-62:
Process ForkPoolWorker-66:
Process ForkPoolWorker-75:
Process ForkPoolWorker-61:
Process ForkPoolWorker-70:
Process ForkPoolWorker-71:
Process ForkPoolWorker-74:
Process ForkPoolWorker-63:
Process ForkPoolWorker-72:
Process ForkPoolWorker-69:
Process ForkPoolWorker-73:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/alejandro/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/alejandro/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/alejandro/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home

KeyboardInterrupt: 

  File "/home/alejandro/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/alejandro/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/alejandro/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/alejandro/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/alejandro/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/alejandro/anaconda3/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/home/alejandro/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/alejandro/anaconda3/lib/python3.8/m

  File "/home/alejandro/anaconda3/lib/python3.8/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
Process ForkPoolWorker-82:
Process ForkPoolWorker-76:
Process ForkPoolWorker-77:
Process ForkPoolWorker-81:
Process ForkPoolWorker-80:
Process ForkPoolWorker-79:
Process ForkPoolWorker-83:
Process ForkPoolWorker-78:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/alejandro/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/alejandro/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/alejandro/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run(